In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy import stats
from scipy.integrate import quad
import pandas as pd

# Case 5: Multi-Look and American Options

## Part 1: Asian Options

In [2]:
S0 = 100 
r = 0.02 
sigma = 0.15
T = 10  # Time horizon
n_steps = 10  # Number of observations
n_sims = 100000  # Number of simulation paths
dt = T/n_steps